# CPI (Consumer Price Index) Explorer

Deep-dive into CPI components, shelter, energy, and special aggregates.

In [ ]:
from macro_econ.series import build_cpi_tree
from macro_econ.clients import FredClient
from macro_econ.transforms import (
    mom_change, mom_annualized, yoy_change, n_month_annualized,
    moving_average, compare_sa_nsa,
    adf_test, kpss_test, stl_decompose, compute_acf_pacf
)
from macro_econ.viz.charts import (
    line_chart, heatmap_table, stl_plot, acf_pacf_plot, recession_shading
)
from macro_econ.viz.widgets import build_tree_widget

fred = FredClient()
cpi = build_cpi_tree()
print(cpi.print_tree())

## CPI Hierarchy Browser

In [ ]:
import ipywidgets as widgets
from IPython.display import display

output = widgets.Output()

def on_node_select(node):
    with output:
        output.clear_output()
        src = node.get_source('fred')
        if src:
            df = fred.fetch_series(src.series_id, start_date='2000-01-01')
            fig = line_chart(
                {node.name: yoy_change(df, 12)},
                f'{node.name} (YoY %)'
            )
            fig = recession_shading(fig)
            fig.show()
        else:
            print(f'No FRED source for {node.name}. Use BLS client for detailed items.')

tree_widget = build_tree_widget(cpi, on_select=on_node_select)
display(tree_widget, output)

## Headline vs Core CPI

In [ ]:
cpi_all = fred.fetch_series('CPIAUCSL', start_date='2000-01-01')
cpi_core = fred.fetch_series('CPILFESL', start_date='2000-01-01')

fig = line_chart(
    {'CPI-U YoY': yoy_change(cpi_all, 12), 'Core CPI YoY': yoy_change(cpi_core, 12)},
    'CPI: Headline vs Core (YoY %)',
    yaxis_title='Percent'
)
fig = recession_shading(fig)
fig.show()

In [ ]:
# Multi-horizon core CPI
fig = line_chart(
    {
        'MoM Ann.': mom_annualized(cpi_core),
        '3m Ann.': n_month_annualized(cpi_core, 3),
        '6m Ann.': n_month_annualized(cpi_core, 6),
        'YoY': yoy_change(cpi_core, 12),
    },
    'Core CPI: Multi-Horizon Inflation Rates'
)
fig.show()

## Shelter Deep-Dive

In [ ]:
oer = fred.fetch_series('CUSR0000SEHC', start_date='2000-01-01')
rent = fred.fetch_series('CUSR0000SEHA', start_date='2000-01-01')

fig = line_chart(
    {
        "Owners' Equivalent Rent YoY": yoy_change(oer, 12),
        'Rent of Primary Residence YoY': yoy_change(rent, 12),
    },
    'Shelter Components (YoY %)',
    yaxis_title='Percent'
)
fig = recession_shading(fig)
fig.show()

## Supercore CPI (Services less Rent of Shelter)

In [ ]:
# Supercore: CPI Services less rent of shelter
# This is a BLS-only series (CUSR0000SASLE), fetch via FRED if available
try:
    supercore = fred.fetch_series('CUSR0000SASLE', start_date='2000-01-01')
    fig = line_chart(
        {
            'Core CPI YoY': yoy_change(cpi_core, 12),
            'Supercore YoY': yoy_change(supercore, 12),
        },
        'Core CPI vs Supercore (Services ex. Shelter) YoY %'
    )
    fig = recession_shading(fig)
    fig.show()
except Exception as e:
    print(f'Supercore not available on FRED: {e}')
    print('Use BlsClient to fetch CUSR0000SASLE directly.')

## Statistical Analysis

In [ ]:
# ADF / KPSS on Core CPI YoY
core_yoy = yoy_change(cpi_core, 12).dropna()

print(adf_test(core_yoy).summary)
print(kpss_test(core_yoy).summary)

# ACF / PACF
acf_vals, pacf_vals = compute_acf_pacf(core_yoy, nlags=24)
fig = acf_pacf_plot(acf_vals, pacf_vals, 'Core CPI YoY: ACF and PACF')
fig.show()

# STL Decomposition
stl_result = stl_decompose(cpi_core['value'], period=12)
fig = stl_plot(stl_result, 'Core CPI: STL Decomposition')
fig.show()